## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-15-01-08-12 +0000,nyt,Live Updates: 15 Victims and a Gunman Dead Aft...,https://www.nytimes.com/live/2025/12/14/world/...
1,2025-12-15-01-06-00 +0000,wsj,Asian Currencies Consolidate Ahead of Major U....,https://www.wsj.com/finance/currencies/asian-c...
2,2025-12-15-01-00-11 +0000,bbc,What we know so far about the Brown University...,https://www.bbc.com/news/articles/ckg9l1j9wxno...
3,2025-12-15-00-58-27 +0000,nyt,California Hires Former C.D.C. Officials Who C...,https://www.nytimes.com/2025/12/14/us/californ...
4,2025-12-15-00-54-35 +0000,nypost,Career shoplifter tries to rob Walmart during ...,https://nypost.com/2025/12/14/us-news/sophia-m...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
101,bondi,28
17,brown,25
102,beach,24
19,shooting,22
18,university,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
109,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,149
47,2025-12-14-22-25-01 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,112
122,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...,108
13,2025-12-15-00-36-03 +0000,latimes,Los Angeles Jewish groups ramp up Hanukkah fes...,https://www.latimes.com/california/story/2025-...,102
127,2025-12-14-15-32-12 +0000,nypost,Bondi Beach terror survivor describes how cops...,https://nypost.com/2025/12/14/world-news/bondi...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
109,149,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
122,56,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...
132,37,2025-12-14-13-40-25 +0000,cbc,"Australian police suspect father, son behind d...",https://www.cbc.ca/news/world/bondi-beach-shoo...
125,30,2025-12-14-15-50-19 +0000,nypost,Trump admits he’s not sure GOP will hold onto ...,https://nypost.com/2025/12/14/us-news/trump-ad...
165,29,2025-12-14-10-00-00 +0000,wsj,The falling out between Rep. Marjorie Taylor G...,https://www.wsj.com/politics/policy/marjorie-t...
8,28,2025-12-15-00-41-20 +0000,wapo,Brown University person of interest was on hig...,https://www.washingtonpost.com
29,28,2025-12-14-23-31-21 +0000,nypost,Unhinged bigot caught on video unleashing anti...,https://nypost.com/2025/12/14/us-news/unhinged...
51,25,2025-12-14-22-16-27 +0000,nypost,Australia’s PM blamed for failing to stand up ...,https://nypost.com/2025/12/14/world-news/austr...
26,24,2025-12-14-23-43-16 +0000,nypost,Holocaust survivors and children among the 15 ...,https://nypost.com/2025/12/14/world-news/holoc...
16,21,2025-12-15-00-16-12 +0000,startribune,Minneapolis rally demands ICE off post office ...,https://www.startribune.com/postal-workers-ral...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
